<html>
<div style="width:100%">
  <div style="width:90%; float:left; background:white; margin:10px">
    <img style ="width:100%"src ="https://raw.githubusercontent.com/publicunpublic/QCMM_Jupyter/main/full.svg">
    <div style="margin-top:2.5%;position:absolute;background-color:#0e4be5;width:6%;height:0.3%"></div>
  </div>
  </div>


<html>
<center>
<br>
<h1 style="color:#E6610F;"> Check Normal Mode of Transition State and verify Nature of TS </h1>
<br><br>
</center>

In [2]:
import qcportal as ptl
import pandas as pd
import numpy as np
from typing import Union
import qcelemental as qcel

## Define Variables

In [50]:
client_address = "152.74.10.245:7778"
ts_opt_spec = 'pbe0-d3_def2-svp'
ts_collection = 'salen-OMe_d-lac_ts_from_scan'
minima_collection = 'salen-OMe_d-lac_minima'
dft_method = 'pbe0'
dft_basis = 'def2-svp'
ts_entry_name = 'ts1_03'
min_1_name = "r"
min_2_name = "i1"
username = None#'svogt'
password = None #"7kyRT-Mrow3jH0Lg6b9YIhEjAcvU9EpFBb9ouMClU5g"

In [51]:
client = ptl.FractalClient(address=client_address, username = username, password = password, verify=False)
client 

FractalClient(server_name='Catalisis Server', address='https://152.74.10.245:7778/', username='None')

## A) Load Modified Psi4 function for NM analysis

In [52]:
def vibanal_wfn(hess: np.ndarray = None, irrep: Union[int, str] = None, molecule=None, energy=None, project_trans: bool = True, project_rot: bool = True, molden=False,name=None, lt=None):
    """Function to perform analysis of a hessian or hessian block, specifically...
    calling for and printing vibrational and thermochemical analysis, setting thermochemical variables,
    and writing the vibrec and normal mode files.
    Parameters
    ----------
    wfn
        The wavefunction which had its Hessian computed.
    hess
        Hessian to analyze, if not the hessian in wfn.
        (3*nat, 3*nat) non-mass-weighted Hessian in atomic units, [Eh/a0/a0].
    irrep
        The irrep for which frequencies are calculated. Thermochemical analysis is skipped if this is given,
        as only one symmetry block of the hessian has been computed.
    molecule : :py:class:`~psi4.core.Molecule` or qcdb.Molecule, optional
        The molecule to pull information from, if not the molecule in wfn. Must at least have similar
        geometry to the molecule in wfn.
    project_trans
        Should translations be projected in the harmonic analysis?
    project_rot
        Should rotations be projected in the harmonic analysis?
    Returns

    -------
    vibinfo : dict
        A dictionary of vibrational information. See :py:func:`~psi4.driver.qcdb.vib.harmonic_analysis`
    """
    from psi4 import geometry, set_memory
    import psi4.core as core
    from psi4.driver import qcdb
    
    if hess is None:
        print("no hessian")
        
    else:
        nmwhess = hess
        
    set_memory('500 MB')
    
    
    print("About to start")
    m=molecule.to_string('xyz')
    mol = geometry(m)
    geom = np.asarray(mol.geometry())
    symbols = [mol.symbol(at) for at in range(mol.natom())]
    vibrec = {'molecule': mol.to_dict(np_out=False), 'hessian': nmwhess.tolist()}

    m = np.asarray([mol.mass(at) for at in range(mol.natom())])
    irrep_labels = mol.irrep_labels()
    
    print(qcdb.vib.__file__)
    print("About to do harmonic analyis...")
    vibinfo, vibtext = qcdb.vib.harmonic_analysis(
        nmwhess, geom, m, None, irrep_labels, dipder=None, project_trans=project_trans, project_rot=project_rot)

    print("Finished harmonic analysis")
   

    if core.has_option_changed('THERMO', 'ROTATIONAL_SYMMETRY_NUMBER'):
        rsn = core.get_option('THERMO', 'ROTATIONAL_SYMMETRY_NUMBER')
    else:
        rsn = mol.rotational_symmetry_number()

    if irrep is None:
        therminfo, thermtext = qcdb.vib.thermo(
            vibinfo,
            T=core.get_option("THERMO", "T"),  # 298.15 [K]
            P=core.get_option("THERMO", "P"),  # 101325. [Pa]
            multiplicity=mol.multiplicity(),
            molecular_mass=np.sum(m),
            sigma=rsn,
            rotor_type=mol.rotor_type(),
            rot_const=np.asarray(mol.rotational_constants()),
            E0=energy) 

        core.set_variable("ZPVE", therminfo['ZPE_corr'].data)  # P::e THERMO
        core.set_variable("THERMAL ENERGY CORRECTION", therminfo['E_corr'].data)  # P::e THERMO
        core.set_variable("ENTHALPY CORRECTION", therminfo['H_corr'].data)  # P::e THERMO
        core.set_variable("GIBBS FREE ENERGY CORRECTION", therminfo['G_corr'].data)  # P::e THERMO

        core.set_variable("ZERO K ENTHALPY", therminfo['ZPE_tot'].data)  # P::e THERMO
        core.set_variable("THERMAL ENERGY", therminfo['E_tot'].data)  # P::e THERMO
        core.set_variable("ENTHALPY", therminfo['H_tot'].data)  # P::e THERMO
        core.set_variable("GIBBS FREE ENERGY", therminfo['G_tot'].data)  # P::e THERMO

    else:
        core.print_out('  Thermochemical analysis skipped for partial frequency calculation.\n')

    
    if molden:
        filename = name+"_"+lt+ ".molden_normal_modes"
        with open(filename, 'w') as handle:
            handle.write(qcdb.vib.print_molden_vibs(vibinfo, symbols, geom, standalone=True))

    return vibinfo, therminfo, symbols, geom

## B) Load  Optimized Transition states from Dataset

In [53]:
ds_opt = client.get_collection("OptimizationDataset", ts_collection)

rr = ds_opt.get_record(ts_entry_name, specification = ts_opt_spec)

if rr.status == ("ERROR" or "INCOMPLETE"):
    print(rr.status ,"Optimization not done yet, please be a little more patient")

else:
    print(rr.status, "Optimization finished!")


RecordStatusEnum.complete Optimization finished!


In [54]:
# Get the optimized TS ID

mol_opt_ts = rr.get_final_molecule()
ts_id = mol_opt_ts.id
mol_opt_ts_init = rr.get_initial_molecule()
ts_id_init = mol_opt_ts_init.id
print("Optimized molecule ID:", ts_id)
print("Initial molecule ID:", ts_id_init)

Optimized molecule ID: 44197
Initial molecule ID: 39776


## C) Get the hessian matrix for the respective transition states from the server

In [55]:
record = client.query_results(driver='hessian', 
                              molecule=ts_id, 
                              method=dft_method, 
                              basis=dft_basis 
                              )

In [56]:
record

[ResultRecord(id='54208', status='COMPLETE')]

## D)  Do harmonic analysis for the hessian matrix 

You can visualize the NM in the .molden_normal_modes file

In [57]:
vib_info = {}
hess_mat = record[0].dict()['return_result']
en = record[0].dict()['properties']['return_energy']

vib_info = vibanal_wfn(hess=hess_mat, molecule=mol_opt_ts, name=ts_entry_name, energy=en, lt=ts_opt_spec, molden=True)

About to start

  Memory set to 476.837 MiB by Python driver.
/home/svogt/.anaconda/envs/qcexplore/lib/python3.7/site-packages/psi4/driver/qcdb/vib.py
About to do harmonic analyis...
Finished harmonic analysis
 '82.1438' '86.0091' '101.5493' '124.6780' '131.0086' '156.2414'
 '160.0250' '168.7885' '176.7204' '190.9213' '217.4765' '221.4203'
 '222.1903' '226.5621' '242.4896' '249.8158' '265.5959' '279.8831'
 '291.2139' '295.3301' '307.4249' '338.8937' '348.5708' '358.4890'
 '364.1902' '366.0807' '394.2702' '406.7257' '415.8547' '419.1181'
 '429.5714' '436.3351' '472.9738' '479.3381' '490.0927' '496.1629'
 '502.3058' '504.1554' '533.6312' '562.6955' '567.0707' '574.8305'
 '598.2637' '614.4077' '622.9391']


## E)  Get Intermadiate candidates from TS normal mode

In [67]:
import math

nm_num = 0  # Number of the normal mode (Imaginary mode is 0)
vib_range = 1.0 # The range of the vibration

vib, therm, symbols, geom = vib_info
nat = int(len(vib['q'].data[:, nm_num]) / 3)
nm = vib['q'].data[:, nm_num].reshape(nat, 3)#[at].real



disp_list = np.arange(-vib_range, vib_range, 0.1)

close_reac = vib_range - 0.1
close_int  = -vib_range


# Saving the extremes as molecule objects
vib_animation = ""
for d in disp_list:
    disp = (geom - d*nm) 
    
    ## Creating molecule from displacements
    mol = qcel.models.Molecule(
    symbols=symbols,
    geometry = disp,
    )
    
    ## Saving reactant and product candidates 
    vib_animation += mol.to_string("xyz")
    if math.isclose(close_int,d):
        min1_cand  = mol
    if math.isclose(close_reac,d):
        min2_cand = mol

    # Saving the 
    filename = "animts2_"+ts_entry_name+"_"+ts_opt_spec+ ".dat"
    with open(filename, 'w') as handle:
        handle.write(vib_animation)

## F) Visualize Intermediate initial structures 

In [68]:
min1_cand

NGLWidget()

In [69]:
mol_opt_ts

NGLWidget()

In [70]:
min2_cand

NGLWidget()

In [71]:
min2_cand.measure([34,43])

3.9495971213500263

## G)  Optimize intermediate states 

In [47]:
#ds = ptl.collections.OptimizationDataset(minima_collection, client=client)
#ds.save()

'177'

In [116]:
ds_opt_min = client.get_collection("OptimizationDataset", minima_collection)

In [117]:
ds_opt_min.add_entry(ts_entry_name+'_'+interm_1_name, min1_cand)
ds_opt_min.add_entry(ts_entry_name+'_'+interm_2_name, min2_cand)

In [118]:
kw = ptl.models.KeywordSet(**{"values": {"dftd": "d3", "convthre" : '3.0e-7', "threall" : '1.0e-13', 'dftgrid' : 2, "scf" : "diis+a"}})
kw_id = client.add_keywords([kw])[0]

method, basis = ts_opt_spec.split('_')

if "d3" in method:
    method = method.split('-')[0]

add_spec = {'name': ts_opt_spec,
        'description': 'Geometric + TERACHEM/'+ts_opt_spec,
        'optimization_spec': {'program': 'geometric', 'keywords': {"converge" : ["set" , "gau_tight"]}},
        'qc_spec': {'driver': 'gradient',
        'method': method,
        'basis': basis,
        'keywords': kw_id,
        'program': 'terachem'}}
ds_opt_min.add_specification(**add_spec,overwrite=True)
ds_opt_min.save()

'177'

In [119]:
c = ds_opt_min.compute(ts_opt_spec, tag='tera')
print(c)

2


## G)  Check Optimization 

In [28]:
ds_opt_min = client.get_collection("OptimizationDataset", minima_collection)
ds_opt_min.status(collapse=False)

,pbe0-d3_def2-svp
ts1_01_r,COMPLETE
ts1_01_i1,COMPLETE
ts1_03_r,COMPLETE
ts1_03_i1,COMPLETE


In [73]:
rr = ds_opt_min.get_record(name='ts1_03_i1', specification = ts_opt_spec)
mol = rr.get_final_molecule()
#print(mol.pretty_print())
mol

NGLWidget()

In [74]:
mol.measure([35,47])

4.203952372755026

In [75]:
rr.show_history()

## Extras

## Load a hessian from a file


In [87]:
#import numpy as np
#h = np.loadtxt("./hessian_1.txt", delimiter = ' ')
#h.shape

### Restart Optimization jobs

In [ ]:
mol_list = []

spec = 'pbe0-d3_def2-svp'

for i in ds_opt.data.records: 
    #print(i)
    mol_list.append(i)

for i in  mol_list:
    opt_rec = ds_opt.get_record(name=i,specification= spec)
    if (opt_rec.status == 'ERROR'):    
        id_u = opt_rec.id 
        print('yes', i, id_u)
        c = client.modify_tasks("modify", new_tag = 'tera',base_result = id_u)
        #c = client.modify_tasks("restart",base_result = id_u)
        print(c)

### Show gradient output

In [66]:
print(rr.get_trajectory()[0].get_stdout())

Startfile from command line: tc.in


       ***********************************************************
       *                    TeraChem v1.94V-2019.08-beta         *
       *                 Hg Version: b27b21c3892d+               *
       *                   Production Version                    *
       *           Chemistry at the Speed of Graphics!           *
       ***********************************************************
       * This program may only be used in connection with        *
       * a valid license from PetaChem, LLC. Use of this program *
       * or results thereof indicates acceptance of all terms    *
       * and conditions stated in the license and that a valid   *
       * license agreement between the user and PetaChem, LLC    *
       * exists. PetaChem, LLC does not warrant the correctness  *
       * of results or their suitability for any purpose.        *
       * Please email bugs, suggestions, and comments to         *
       *                 

### Show Optimization output

In [65]:
print(rr.get_stdout())

QCEngine selected.
174 internal coordinates being used (instead of 174 Cartesians)
Internal coordinate system (atoms numbered from 1):
Distance 1-2
Distance 1-6
Distance 1-34
Distance 2-3
Distance 2-30
Distance 3-4
Distance 3-20
Distance 4-5
Distance 4-7
Distance 5-6
Distance 5-28
Distance 6-32
Distance 7-8
Distance 7-26
Distance 8-9
Distance 8-35
Distance 9-10
Distance 9-23
Distance 9-24
Distance 10-11
Distance 10-21
Distance 10-22
Distance 11-12
Distance 11-35
Distance 12-13
Distance 12-25
Distance 13-14
Distance 13-18
Distance 14-15
Distance 14-19
Distance 15-16
Distance 15-29
Distance 16-17
Distance 16-33
Distance 17-18
Distance 17-31
Distance 18-27
Distance 27-35
Distance 28-35
Distance 35-36
Distance 35-43
Distance 36-37
Distance 37-38
Distance 37-39
Distance 37-40
Distance 41-45
Distance 41-47
Distance 42-46
Distance 42-48
Distance 43-47
Distance 44-48
Distance 45-48
Distance 45-49
Distance 45-51
Distance 46-47
Distance 46-50
Distance 46-52
Distance 49-53
Distance 49-54
Distance

### Visualize optimization trayectory (Molden)

In [123]:
from pathlib import Path

tr_mols = rr.get_molecular_trajectory()
tr_grads = rr.get_trajectory()

molden_str = ''
for i in range(len(tr_mols)):
    enr = tr_grads[i].dict()['properties']['return_energy']
    molden_str += tr_mols[i].to_string(dtype='xyz').replace("0 1 "+tr_mols[i].get_molecular_formula(), str(enr))

opt_file = Path("./opt_trj_"+'ts1_01_i1'+'.dat')

with opt_file.open("w", encoding ="utf-8") as f:
    f.write(molden_str)

In [102]:
#Delte Entry 

#del ds_opt_min.data.records['ts1_02_i1']
#ds_opt_min.save()

'177'